# Data

In [ ]:
from google.colab import drive
import random
from torch import nn
import torch
import nltk
import string
import re
import numpy as np
import pandas as pd

import torch
import torchtext
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, TensorDataset

from collections import Counter

drive.mount('/content/drive/')
data_path = '/content/drive/Shareddrives/DeepLearning_2022/Final-Project/input/'

Mounted at /content/drive/


In [ ]:

def preprocessing_text(text,n = True):
  #To lowecase the text
  result = text.lower()
  #Eliminate digits
  result = re.sub(r'\d+', '', result)
  #Eliminate \n
  result = re.sub("\n\n","",result)
  if n:
    result = re.sub("\n"," ",result)
  #Considering \n as one more word
  else:
    result = re.sub("\n"," \n ",result)
  #Eliminate \t
  result = re.sub("\t"," ",result)

  #Eliminate any punctuation and symbol
  result = re.sub("[\[\]\(\)\"=!º.,¿?;:«»*_]",' ',result)
  #Eliminate multiple white spaces
  result = re.sub(' +', ' ', result)

  return result

def vocabulary(words):
  """
  Given a plain text in a list returns a list with all the different words.
  """
  vocab = set()
  for word in words:
    if word not in vocab:
      vocab.add(word)

  return list(vocab)

#Decide what to do with words that are not in the vocabulary
def text_to_seq(text,vocab,add_n = False):
  """
  Given a text split it and a vocabulary, return a list of numbers that refers 
  to the index position of each word in the vocabulary.
  """
  sequence = []
  for word in text:
    if word != "":
      sequence.append(vocab.index(word))
  if add_n:
    sequence.append(vocab.index("\n"))

  return torch.tensor(sequence)



In [ ]:
with open(data_path + "poesia_v3.txt","r") as f:
  poesia = ""
  for line in f:
    poesia += line

poe = re.sub("--","%",poesia)
poesias_list = poe.split("%")

poesies_proc = []
poesies_vocab = []
for poem in poesias_list:
  poesies_vocab.append(preprocessing_text(poem))
  poesies_proc.append(preprocessing_text(poem,False))

words = " ".join(poesies_proc).split(" ")
vocab = vocabulary(words)[1:]
print("Number of poems",len(poesias_list))
print("Number of words in total",len(words))
print("Number of words of the vocabulary",len(vocab))


Number of poems 290
Number of words in total 35034
Number of words of the vocabulary 7554


## Defining the training data (n-grams)

In [ ]:
def data_process(corpus, vocab):
    data = list()
    for text in corpus:
      for paragraph in text.split(" \n "):
        token_list = text_to_seq(paragraph.split(" "),vocab,True)
        for i in range(1, len(token_list)):
            n_gram_seq = torch.tensor(token_list[:i+1], dtype=torch.long)
            data.append(n_gram_seq)
    return data

In [ ]:
train_data = data_process(poesies_proc, vocab)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


In [ ]:
X = [i[:-1] for i in train_data]   # taking all the words except the last in the input set
y = [i[-1] for i in train_data]    # taking last words in the output set
print("X shape",len(X))
print("y shape",len(y))
print(X[0],y[0])

X shape 30331
y shape 30331
tensor([6049]) tensor(7387)


In [ ]:

y = torch.from_numpy(np.array(y))
print(y)

tensor([7387, 6404, 4116,  ...,  378, 6427, 3729])


Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:

class LSTM_model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,dropout):
        super(LSTM_model, self).__init__()
        self.embed = torch.nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(input_size, hidden_size)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        #self.sigm = nn.Sigmoid()
        #self.dropout = nn.Dropout(dropout)

    def zero_hidden(self):
        # Initialize lstm states with zeros
        return (torch.zeros(1, 1, hidden_dim).cuda(),
                torch.zeros(1, 1, hidden_dim).cuda())

    def forward(self, input):
        x = self.embed(input).unsqueeze(1) #We need the unsqueze to introduce the new dimension
        output_lstm, states_lstm = self.lstm1(x, self.zero_hidden())
        output_lstm, states_lstm = self.lstm2(output_lstm, self.zero_hidden())
        output_lstm, states_lstm = self.lstm3(output_lstm, self.zero_hidden())
        #output_lstm = self.dropout(output_lstm)
        out = self.fc(output_lstm)
        #out = self.sigm(out)
        return out, output_lstm, states_lstm

In [ ]:
num_epochs = 20
embedding_dim = 400
hidden_dim = 250
vocab_size = len(vocab)

LSTM_model = LSTM_model(embedding_dim, hidden_dim, vocab_size,dropout=None).cuda()

CE_loss = torch.nn.CrossEntropyLoss() 
#loss = nn.NLLLoss()

optimizer_LSTM = torch.optim.Adam(list(LSTM_model.parameters()), lr=0.0000005,weight_decay=0.001)

In [ ]:
def train_loop(model, num_epochs, CE_loss, optimizer, X,y):

    loss_hist = []
    min_loss = float("inf")

    for x in range(num_epochs):
        print('Epoch: {}'.format(x))
        optimizer.zero_grad()
        # TRAINING LOOP
        for i,instance in enumerate(X):
            val = instance.cuda()
            #y_aux = np.zeros(len(vocab))
            #y_aux[int(y[i])] = 1.0
            y_val = y[i].cuda()
            #y_val = torch.tensor(y_aux).cuda()
            y_hat, _, _ = model(val)
            # Calculate loss
            #loss = CE_loss(y_hat[0][0], y_val)
            loss = CE_loss(y_hat[0][0], y_val)
            # Backpropagate
            loss.backward()
            #Clip gradient
            #nn.utils.clip_grad_norm_(model.parameters(), 5)
            # Update weights
            optimizer.step()
        if loss.item() < min_loss:
          min_loss = loss.item()
          torch.save(LSTM_model.state_dict(), data_path+'/poem_generator_minloss.ckpt')
        print('Loss: {:6.4f}'.format(loss.item()))
        loss_hist.append(loss.item())
    return loss_hist

In [ ]:
loss_hist = train_loop(LSTM_model, num_epochs, CE_loss, optimizer_LSTM, X, y)

In [ ]:
torch.save(LSTM_model.state_dict(), data_path+'/poem_generator_0.4222.ckpt')

In [ ]:
LSTM_model.load_state_dict(torch.load(data_path + '/poem_generator_minloss.ckpt'))

<All keys matched successfully>

# Generator

In [ ]:
def tensor_to_str(tensor,vocab):
  text = ""
  for elem in tensor:
    text += " "+ vocab[int(elem)]
  return text

In [ ]:
sig = nn.Sigmoid()

In [ ]:
def generator(model,n_words,vocab,seed):
  result = text_to_seq(seed.split(" "),vocab).cuda()
  final_list = list(result)
  for n in range(n_words):
    #Instead of max sample
    #print(torch.tensor(final_list))
    val = model(torch.tensor(final_list).cuda())[0][0]
    val = sig(val)
    final_list.append(val.multinomial(num_samples=1))
    #final_list.append(torch.argmax(val))
    #print(tensor_to_str(final_list,vocab))
  #print(torch.tensor(final_list))
  return tensor_to_str(final_list,vocab)
seed = "jo sóc"
print(generator(LSTM_model,3,vocab,seed))

 jo sóc d'orgull mai ciutats


In [ ]:
#Improved model

#la mort camí els déu com raó rica perquè ran balanguera ens més plors plans qui bellesa és 
# dona l'amor de

# la vida dirà m'enjoio dirà esvaïa brisa llibertat opulenta llibertat lírica realitat somni sóc naixença déu de sang pel goig anar semblaria
#la vida frem carrer infidels encara resen mort negre sofisticcada assassines negui creient-me aigua pirineus les davant vençó enveja trontollar platja feria

In [ ]:
#First model

#la mort em son sa mal l'hora veu si d'amor enrera estaig no tan ho dolç és fent ha pugui ira dolç un

#la mar dolça les amor els al es sense anglès dels ben blanc si es tot bocí jo si és tot teu paraula

#paraules institució alts al gent és i tu com e us 
#home vent al no jo quan perquè més si

#paraules font somni joia com perd per si o hi honor ulls plors les goig amb que en marriments l'hora sense

#la mar ens mira d'amor ja l'hora és crit pregon una del nom ens es veu molt a un més un camí veu de déu cor hom del i la déu amb ara les

In [ ]:
r,_,_ = LSTM_model(X[10].cuda())
vals = r[0][0].cpu().detach().numpy()
#( r[0][0] - torch.min(r[0][0]) ) / ( torch.max(r[0][0]) - torch.min(r[0][0]) )
#(vals - np.min(vals)) / (np.max(vals) - np.min(vals))
r[0][0].multinomial(num_samples=1)
#torch.argmax(r[0][0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([345], device='cuda:0')

In [ ]:
a = r[0][0]
b = np.zeros(len(vocab),dtype=np.float32)
b[int(y[10])] = float(1.0)
b = torch.tensor(b).cuda()

In [ ]:
torch.log(a).dtype

torch.float32

In [ ]:
b

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')

In [ ]:
loss(torch.log(a), b)

RuntimeError: ignored

In [ ]:
#y[10]
a = np.zeros(len(vocab))
a[1528] = 1
tn = torch.tensor(a)
print(tn.cuda())
print(r[0][0])

tensor(1., device='cuda:0', dtype=torch.float64)
tensor([2.0237e-07, 2.5471e-05, 2.8484e-04,  ..., 7.8355e-05, 2.7058e-07,
        3.2904e-01], device='cuda:0', grad_fn=<SelectBackward0>)


In [ ]:

class LSTM_model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,dropout):
        super(LSTM_model, self).__init__()
        self.embed = torch.nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(input_size, hidden_size)
        self.lstm2 = nn.LSTM(hidden_size, (hidden_size//2))
        self.fc = nn.Linear((hidden_dim//2), output_size)
        #self.dropout = nn.Dropout(dropout)

    def zero_hidden(self,n_lstm,n_lstm2):
        # Initialize lstm states with zeros
        return (torch.zeros(1, 1, hidden_dim//n_lstm).cuda(),
                torch.zeros(1, 1, hidden_dim//n_lstm2).cuda())

    def forward(self, input):
        x = self.embed(input).unsqueeze(1) #We need the unsqueze to introduce the new dimension
        output_lstm, states_lstm = self.lstm1(x, self.zero_hidden(1,1))
        #output_lstm = self.dropout(output_lstm)
        output_lstm, states_lstm = self.lstm2(output_lstm, self.zero_hidden(2,2))
        #output_lstm = self.dropout(output_lstm)
        output_fc = self.fc(output_lstm)
        return output_fc, output_lstm, states_lstm